In [ ]:
import pandas as pd

# Predicting Authorship of the Disputed Federalist Papers

The Federalist Papers are a collection of **85 essays** written by James Madison, Alexander Hamilton, and John Jay under the collective pseudonym "Publius" to promote the ratification of the United States Constitution.

Authorship of most of the papers were revealed some years later by Hamilton, though his claim to authorshipt of 12 papers were disputed for nearly 200 years (studies generally agree that the disputed essays were written by James Madison.)

| Author | Papers |
| :- | -: | 
| Jay | 2, 3, 4, 5, 64
| Madison | 10, 14, 37-48
| Hamilton | 1, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, 21-36, 59, 60, 61, 65-85
| Hamilton and Madison | 18, 19, 20
| Disputed | 49-58, 62, 63

The goal of this problem is to train a classifier that predicts the author of the disputed papers.

In [ ]:
# load Federalist papers data
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/papers.csv'
data = pd.read_csv(url)
data.head()

In [ ]:
# Federalist paper No. 1
print(data.paper[0])

In [ ]:
data.author.value_counts()

**Part 1 (text processing):** remove stop words and punctuations from the papers, and lemmatize them.

In [ ]:
# remove stop words and punctuations
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stop_words.update(punctuations)

def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)

data['paper'] = data['paper'].apply(remove_stopwords)

# lemmatize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    word_tokens = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
    return ' '.join(lemmatized_text)

data['paper'] = data['paper'].apply(lemmatize)
data

**Part 2: train-test split**

We'll use the papers written by Hamilton and Madion as the training set, and the disputed papers as the testing set.

In [ ]:
data_train = data[data.author.isin(['Hamilton','Madison'])]
data_test = data[data.author=='Disputed']

Extract feature matrices X_train and X_test, and target vector y_train

In [ ]:
X_train = data_train.paper
X_test = data_test.paper
y_train = data_train.author

**Part 3:** build a classification pipeline (count vectorizer + Naive Bayes model) that predicts the author of a paper.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer()),
                        ('clf', MultinomialNB())])

text_clf.fit(X_train, y_train)

**Part 4:** Use a grid search to tune the pipeline hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
                'clf__alpha': (0.1, 1, 10)}

gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)
best_clf = gs_clf.best_estimator_
best_clf.fit(X_train, y_train)

**Part 5:** How does your classification model choose between Hamilton and Madison?

In [23]:
X_train_words = best_clf.named_steps['vect'].get_feature_names_out()
# number of times each word appears across all Hamilton's papers
Hamilton_word_count = best_clf.named_steps['clf'].feature_count_[0,:]
# number of times each word appears across all Madison's papers
Madison_word_count = best_clf.named_steps['clf'].feature_count_[1,:]
# create a DataFrame of words with their separate Hamilton and Madison counts
words = pd.DataFrame({'word' : X_train_words, 'Hamilton' : Hamilton_word_count, 'Madison' : Madison_word_count}).set_index('word')
words.Hamilton = words.Hamilton+1
words.Madison = words.Madison+1
words.Hamilton = words.Hamilton/words.Hamilton.sum()
words.Madison = words.Madison/words.Madison.sum()
# Add ratios
words['Hamilton_ratio'] = words.Hamilton/words.Madison
words['Madison_ratio'] = words.Madison/words.Hamilton
words.sort_values(by='Hamilton_ratio', ascending=False).head(10)
words.sort_values(by='Madison_ratio', ascending=False).head(10)

,Hamilton,Madison,Hamilton_ratio,Madison_ratio
word,,,,
democracy,0.000016,0.000499,0.032752,30.532944
provisions,0.000016,0.000384,0.042577,23.486880
had,0.000033,0.000615,0.053221,18.789504
assumed,0.000016,0.000307,0.053221,18.789504
felony,0.000016,0.000269,0.060824,16.440816
indirectly,0.000016,0.000269,0.060824,16.440816
alterations,0.000016,0.000269,0.060824,16.440816
whilst,0.000033,0.000499,0.065503,15.266472
novelty,0.000016,0.000231,0.070962,14.092128


**Part 6:** use your classifier to find who was the most likely author of the 12 disputed essays: Hamilton or Madison.

In [25]:
# predict author of disputed papers
y_pred = best_clf.predict(X_test)
# print results as a nice dataframe
results = pd.DataFrame({'paper' : data_test.paper, 'author' : y_pred}).set_index('paper')
results

author
paper                                                       
To People State New York THE author `` Notes St...   Madison
To People State New York IT MAY contended perha...   Madison
To People State New York TO WHAT expedient shal...   Madison
To People State New York FROM general inquiry p...  Hamilton
To People State New York I SHALL perhaps remind...  Hamilton
To People State New York THE next view I shall ...   Madison
To People State New York THE number House Repre...  Hamilton
To People State New York THE SECOND charge Hous...  Hamilton
To People State New York THE THIRD charge House...  Hamilton
To People State New York THE remaining charge H...  Hamilton
To People State New York HAVING examined consti...  Hamilton
To People State New York A FIFTH desideratum il...   Madison